# JSON Path Expressions
Updated: 2019-10-03

JSON documents have an inherent structure to them, similar to XML documents or a file system. Many of the JSON functions provided with Db2 need a method to navigate through a document to retrieve the object or item that the user wants. To illustrate how a JSON path expression points to a particular object, one of the customer documents will be used:
```json
{
    "customerid": 100000,
    "identity":  {
                  "firstname": "Jacob",
                  "lastname": "Hines",
                  "birthdate": "1982-09-18"
                 },
    "contact":   {
                  "street": "Main Street North",
                  "city": "Amherst",
                  "state": "OH",
                  "zipcode": "44001",
                  "email": "Ja.Hines@yahii.com",
                  "phone": "813-689-8309"
                 },
    "payment":   {
                  "card_type": "MCCD",
                  "card_no": "4742-3005-2829-9227"
                 },
    "purchases": [
                   {
                    "tx_date": "2018-02-14",
                    "tx_no": 157972,
                    "product_id": 1860,
                    "product": "Ugliest Snow Blower",
                    "quantity": 1,
                    "item_cost": 51.86
                   }, …additional purchases…
                 ]
}
```

### Document Structure
Every JSON path expression begins with a dollar sign (`$`) to represent the root or top of the document structure. To traverse down the document, the dot/period (`.`) is used to move down one level. The "`$`" and "`.`" characters are reserved characters for the purposes of path expressions so care needs to be taken not to use them as a part of a key name in a key-value pair.

In our customer document example, to refer to the value associated with the customerid key, the path expression would be:
```json
$.customerid
```
To retrieve the value associated with the identity key, the path expression would be:
```json
$.identity
```
The value referred to in this last example is the entire JSON object that is the value associated with identity so the following object would be returned:
```json
{
   "firstname": "Jacob",
   "lastname" : "Hines",
   "birthdate": "1982-09-18"
}
```
If we needed to traverse the interior of the JSON OBJECT value associated with identity, for example to refer to the birthdate, then we would append the initial key name with a period and the internal key name for the value of interest:
```json
$.identity.birthdate
Result: "1982-09-18"
```
Up to this point, we have only been referring to objects and individual key-value pairs within a JSON document. JSON allows for the use of simple arrays (such as phone numbers) or for arrays of objects (like purchases). 
A simple and complex array type are shown below:
```json
{
  "employee": 10,
  "phoneno": ["592-533-9042","354-981-0032","919-778-1539"]
}

{
  "division": 5,
  "stores" : 
    [
      {"id": 45, "city" : "Toronto"},
      {"id": 13, "city" : "Markham"},
      {"id": 93, "city" : "Schaumburg"}
    ]
}
```
To refer to an entire array you would just reference the object name.
```json
$.phoneno

Result: ["592-533-9042","354-981-0032","919-778-1539"]
```
To reference the first element of an array, you would append an array specifier (square brackets `[]`) with the number representing the position of the element in the array inside the brackets. This number is also referred to as the array index value, or simply the index value. The first element of a JSON array always begins at index value zero. 
To refer to the first phone number from the above list, we would use this path:
```json
$.phoneno[0]

Result: "592-533-9042"
```
An index value must be provided, otherwise the Db2 JSON functions will return a null (or an error depending on other settings which we have not yet discussed). The reverse situation, when you specify an index value of zero for a non-array field will cause an error in strict mode and be accepted under lax mode with the contents of the field will be returned. For instance, the following path expression is acceptable to Db2 JSON functions when using lax mode:
```json
$.stores[0].city[0]

Result: "Toronto"
```
A good development practice would be to define how fields should be created in a document. If a field could potentially have multiple values, then single items should be inserted as arrays with a single value rather than as an atomic item. Examining the document will make it clear that an object could have more than one item in it such as in this example:
```json
"phoneno": ["592-533-9042"]
```
Dealing with arrays of objects is similar to simple objects – an index value is used before traversing down the document. To retrieve the city of the second store of the division would require the following path statement:
```json
$stores[1].city

Result: "Markham"
```
Since stores is an array of objects, we must first select which object in the array needs to be retrieved. The `[1]` represents the second object in the array:
```json
{"id": 13, "city" : "Markham"}
```
We used the dot notation to traverse the contents of the object to refer to the city. Arrays can be nested to many levels and can make path expressions complex. 

The next document has two levels of arrays.
```json
{
    "division": 5,
    "stores" : 
    [
       {"id": 45, "phone": ["592-533-9042","354-981-0032"]},
       {"id": 13, "phone": ["634-231-9862"]},
       {"id": 93, "phone": ["883-687-1123","442-908-9435","331-991-2433"]}
    ]
}
```
To create a path to the second phone number of store 93, we would need to use two array specifications:
```json
$.stores[2].phone[1]
```
As the depth of the path expression increases, the potential for errors also becomes higher. One way of reducing the complexity of path expressions is to use two or more steps to traverse a document. For the example above, a user could use the following approach:
* stores = using(document) find the contents of `$.stores[2]`
* result = using(stores) find the contents of `$.phone[1]`

### Path Expression Summary
The following table summarizes the examples of path expressions that have been covered so far (using the division/stores example above).

|Pattern | Result
|:-------|:------
| `$.division` | `5`
| `$.stores`| `[{"id": 45, "phone": ["592-533-9042","354-981-0032"]},`
||  `{"id": 13, "phone": ["634-231-9862"]},`
||  `{"id": 93, "phone": ["883-687-1123","442-908-9435","331-991-2433"]}]`
| `$.stores[0]` | `{"id": 45, "phone": ["592-533-9042","354-981-0032"]}`
| `$.stores[1].id` | `13`
| `$.stores[2].phone` | `["883-687-1123","442-908-9435","331-991-2433"]`
| `$.stores[2].phone[1]` | `"442-908-9435"`

### Simplifying JSON Path Expressions
The previous section illustrated two shortcomings of JSON path expressions:
* JSON path expressions can get complex, especially when dealing with arrays and objects within objects
* Path expressions are limited to referencing only an individual object, array, or item

When writing path expressions, the potential for spelling mistakes goes up as the path gets longer! If the field name is unique in a document, it can be referred to much more easily by using the asterisk (`*`) or wildcard character.

The wildcard character (asterisk `*`) can be used to match any object in a level or an array. The asterisk does not match all levels in the document, just the immediate one.

For instance, consider the following document:
```json
{
  "employee": 10,
  "details" : 
    {
      "name": 
        {
          "first":"George",
          "last" :"Baklarz"
        },
      "phoneno": ["592-533-9042","354-981-0032","919-778-1539"]
    }
}
```
To refer to the last name of the individual in the document, we could write the following path expression:
```json
$.details.name.last

Result: Baklarz
```
The asterisk can be used to match anything at the current level. The equivalent path expression is:
```json
$.*.*.last

Result: Baklarz
```
This technique is useful when the key is unique but can cause problems when the key is duplicated throughout the document. The next section discusses the use of the wildcard character to retrieve multiple values and the pitfalls associated with it.

### Referring to Multiple Objects with JSON Path Expressions
In some situations, a developer may want to retrieve all objects within a document that have the same key name. JSON path expressions include the option of using the asterisk character (`*`) to match any name at the current level. If there are multiple objects that match, then all the matches will be returned.
The following document has multiple objects with the same name. From a development perspective, it doesn't make any sense to use different keywords for first and last names in a document, so this is a reasonable naming convention. 
```json
{
  "authors": 
    {
      "primary"   : {"first_name": "Paul",  "last_name" : "Bird"},
      "secondary" : {"first_name": "George","last_name" : "Baklarz"}       
    },
  "foreword": 
    {
      "primary"   : {"first_name": "Thomas","last_name" : "Hronis"},
      "formats": ["Hardcover","Paperback","eBook","PDF"]
    }
}
```
The following table summarizes to what an asterisk in the path expression is referring to.

|Pattern |Path |Result
|:----|:-----|:------
|`$.authors.primary.last_name` |`$.authors.primary.last_name` |`Bird`
|`$.*.primary.last_name`|`$.authors.primary.last_name`|`Bird, Hronis`
| |`$.foreword.primary.last_name`
|`$.*.*.last_name`|`$.authors.primary.last_name` | `Bird, Baklarz, Hronis`
| | `$.authors.secondary.last_name`
| | `$.foreword.primary.last_name`
|`$.authors.*.last_name` |`$.authors.primary.last_name`|`Bird, Baklarz`
| |`$.authors.secondary.last_name`

As you can tell from the examples that there are drawbacks when using asterisk in a pattern:
* The relationship of the item (last_name) within the document is unknown (i.e. what was the field part of?)
* One or more last_name fields names could be returned, which means that the JSON function using this path needs to be able to handle more than one value (i.e. you can’t use a JSON scalar function, which expects to deal with one value, to handle multiple values. You will get an error!) 

A developer needs to be aware of these limitations when using wildcard expressions.
The wildcard character can also be used in two other ways:
* To refer to all elements in an array
* To refer to all values in an object

Using the wildcard character in an array specification allows the JSON path expression to retrieve the individual values that are in each array element. This is primarily used for arrays that contain objects rather than atomic values.

The author example has been modified to make the author list into an array.
```json
{
  "authors": 
    [
      {"first_name": "Paul",  "last_name" : "Bird"},
      {"first_name": "George","last_name" : "Baklarz"}
    ],
  "foreword": 
    {
      "primary": {"first_name": "Thomas","last_name" : "Hronis"}
    },
  "formats": ["Hardcover","Paperback","eBook","PDF"]
}
```
Referring to all of the book formats in the document can be achieved using one of these two techniques:
```json
$.formats
$.formats.*
```
In the first case, a single JSON array is returned which consists of an array of strings:
```json
["Hardcover","Paperback","eBook","PDF"]
```
The second statement returns 4 individual values:
```json
"Hardcover","Paperback","eBook","PDF"
```
The wildcard character could also be placed at the end of a JSON path expression to retrieve all values in an object. The following path expression refers to all of the values in the foreword author. 
Note that the keys (first_name, last_name) are not retrieved.
```json
$.foreword.primary.* 

Result: "Thomas","Hronis"
```
If you wanted to retrieve all last_names in the authors array, you would use the following path expression:
```json
$.authors[*].last_name

Result: ["Bird","Baklarz"]
```
The use of the wildcard character can be very powerful when dealing with JSON path expressions. The user must take care to ensure that the results being returned are from the appropriate level within the document since the path expression does not recurse into the document.

The following table summarizes what an asterisk in an array specification and at the end of a JSON path expression would produce.

| Pattern | Result
|:--------|:----------
|`$.authors` | `[{"first_name":"Paul",  "last_name":"Bird"},` 
| | `{"first_name":"George","last_name":"Baklarz"}]`
|`$.authors.*` | `"Paul","Bird","George","Baklarz"`
|`$.authors[*].last_name` | `"Bird", "Baklarz"`
|`$.foreword.primary.*` | `"Thomas", "Hronis"`
| `$.formats` | `["Hardcover","Paperback","eBook","PDF"]`
|`$.formats[*]` | `"Hardcover","Paperback","eBook","PDF"`

In summary, a JSON path expression can be used to navigate to individual elements, objects, arrays, or allow for multiple matches within a document. 

### Summary
The following list summarizes how a JSON path expression is built.
* The top of any path expression is the anchor symbol (`$`)
* Traverse to specific objects by using the dot operator (`.`)
* Use square brackets `[]` to refer to items in an array with the first item starting at zero 
* Use the backslash `\` as an escape character when key names include any of the JSON path characters `(.,*,$,[,])`
* Use the asterisk (`*`) to match any object at the current level
* Use the asterisk (`*`) to match all objects in an array or retrieve only the value fields from an object

#### Credits: IBM 2019, George Baklarz [baklarz@ca.ibm.com]